# 来源

** RL **:
    
    介绍：http://karpathy.github.io/2016/05/31/rl/
    
    代码：https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5

    中文：https://www.jiqizhixin.com/articles/2016-06-15

** gym教程：**
    
    https://www.cnblogs.com/mandalalala/p/6227201.html

# gym 初步了解

In [21]:
from matplotlib import pyplot as plt
# plt.ion()
%matplotlib inline
import gym
env = gym.make('Pong-v0')
init_observ = env.reset() #重置环境的状态，返回observations。
print(init_observ.shape)
print(env.action_space) # 查看这个环境中可用的 action 有多少个
print(env.observation_space.shape)    # 查看这个环境中可用的 state 的 observation 有多少个

(210, 160, 3)
Discrete(6)
(210, 160, 3)


In [20]:
# 每一个游戏不同, 每个observation 不同
env = gym.make('CartPole-v0')
print(env.action_space) # 查看这个环境中可用的 action 有多少个
print(env.observation_space.shape)    # 查看这个环境中可用的 state 的 observation 有多少个

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Discrete(2)
(4,)


In [25]:
env = gym.make('CartPole-v0')
for i_episode in range(10): # 玩10次
    observation = env.reset()
    for t in range(100):    # 每次至少100个时间步长，100个时间步长也有可能游戏没有结束哦。。。
        env.render()        # 刷新当前环境，并显示
        print(observation)  # 打印每次的观察值
        action = env.action_space.sample()  # 随机选取一个动作
        observation, reward, done, info = env.step(action) # step 根据下一步的action，返回： 观察，奖励，是否结束一个环节，调试信息。
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[ 0.02490172 -0.03854049 -0.02606066  0.02035432]
[ 0.02413091 -0.2332792  -0.02565357  0.30470223]
[ 0.01946533 -0.42802635 -0.01955953  0.58918551]
[ 0.0109048  -0.62286903 -0.00777582  0.8756435 ]
[-0.00155258 -0.81788442  0.00973705  1.16587167]
[-0.01791027 -1.01313174  0.03305448  1.46159142]
[-0.0381729  -1.2086429   0.06228631  1.7644142 ]
[-0.06234576 -1.01427796  0.0975746   1.49173249]
[-0.08263132 -0.8204696   0.12740925  1.23104419]
[-0.09904071 -0.62719563  0.15203013  0.9808402 ]
[-0.11158463 -0.82399207  0.17164693  1.31715485]
[-0.12806447 -1.02081835  0.19799003  1.65827243]
Episode finished after 12 timesteps
[ 0.01847444  0.03164019 -0.04153943  0.02817555]
[ 0.01910724 -0.16286219 -0.04097592  0.30746837]
[ 0.01585    -0.35737704 -0.03482656  0.58695218]
[ 0.00870246 -0.55199437 -0.02308751  0.86846422]
[-0.00233743 -0.35656604 -0.00571823  0.56861272]
[-0.00946875 -0

# RL 

In [4]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
import numpy as np
import pickle
import gym

# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = True # resume from previous checkpoint?
render = False

# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
    model = pickle.load(open('save.p', 'rb'))
else:
    model = {}
    model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization 权重使用xavier初始化
    model['W2'] = np.random.randn(H) / np.sqrt(H)
  
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

In [2]:
def sigmoid(x): 
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
    """ 处理图片,裁剪,下采样,改变大小,变换为0和1"""
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop 
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

def policy_forward(x):
    """ 前向传播, 两层的策略网络"""
    h = np.dot(model['W1'], x)
    h[h<0] = 0 # ReLU nonlinearity
    logp = np.dot(model['W2'], h)
    p = sigmoid(logp)
    return p, h # return probability of taking action 2, and hidden state

def policy_backward(eph, epdlogp):
    """ backward pass. (eph is array of intermediate hidden states) """
    dW2 = np.dot(eph.T, epdlogp).ravel()
    dh = np.outer(epdlogp, model['W2'])
    dh[eph <= 0] = 0 # backpro prelu
    dW1 = np.dot(dh.T, epx)
    return {'W1':dW1, 'W2':dW2}

In [3]:
np.random.uniform()

0.772770632091281

In [7]:
a=[1,2,3,4,5,6]
b=[100,102,103]
b = np.vstack(a)
b.shape

(6, 1)

In [5]:
env = gym.make("Pong-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0
while True:
  if render: env.render()

  # preprocess the observation, set input to network to be difference image. 两帧画面的差
  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x

  # forward the policy network and sample an action from the returned probability 输入到策略网络并得到下一步动作
  aprob, h = policy_forward(x)
  action = 2 if np.random.uniform() < aprob else 3 # roll the dice! 随机选择一个动作,因为目的是为了采样来进行训练。确实有用？

  # record various intermediates (needed later for backprop)
  xs.append(x) # observation
  hs.append(h) # hidden state
  y = 1 if action == 2 else 0 # a "fake label"  # 假的标签？ 有用？
  dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)
    # grad of log(p)
    
  # step the environment and get new measurements
  observation, reward, done, info = env.step(action)
  reward_sum += reward

  drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

  if done: # an episode finished
    episode_number += 1

    # stack together all inputs, hidden states, action gradients, and rewards for this episode
    epx = np.vstack(xs)
    eph = np.vstack(hs)
    epdlogp = np.vstack(dlogps)
    epr = np.vstack(drs)
    xs,hs,dlogps,drs = [],[],[],[] # reset array memory

    # compute the discounted reward backwards through time
    discounted_epr = discount_rewards(epr)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_epr -= np.mean(discounted_epr)
    discounted_epr /= np.std(discounted_epr)

    epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
    grad = policy_backward(eph, epdlogp)
    for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

    # perform rmsprop parameter update every batch_size episodes
    if episode_number % batch_size == 0:
      for k,v in model.items():
        g = grad_buffer[k] # gradient
        rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
        model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
        grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

    # boring book-keeping
    running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
    print('resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward))
    if episode_number % 100 == 0: pickle.dump(model, open('save.p', 'wb'))
    reward_sum = 0
    observation = env.reset() # reset env
    prev_x = None

  if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
    print(('ep %d: game finished, reward: %f'%(episode_number, reward)) + ('' if reward == -1 else ' !!!!!!!!'))

ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, rewar

ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
ep 9: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.951478
ep 10: game finished, reward: -1.000000
ep 10: game finished, reward: -1.000000
ep 10: game finished, reward: -1.000000
ep 10: game finished, reward: -1.000000
ep 10: game finished, 

ep 17: game finished, reward: -1.000000
ep 17: game finished, reward: -1.000000
ep 17: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.907769
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: 1.000000 !!!!!!!!
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: 1.000000 !!!!!!!!
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, reward: -1.000000
ep 18: game finished, rewar

ep 26: game finished, reward: -1.000000
ep 26: game finished, reward: 1.000000 !!!!!!!!
ep 26: game finished, reward: -1.000000
ep 26: game finished, reward: -1.000000
ep 26: game finished, reward: -1.000000
ep 26: game finished, reward: -1.000000
ep 26: game finished, reward: -1.000000
ep 26: game finished, reward: -1.000000
ep 26: game finished, reward: -1.000000
ep 26: game finished, reward: -1.000000
ep 26: game finished, reward: -1.000000
ep 26: game finished, reward: -1.000000
ep 26: game finished, reward: -1.000000
ep 26: game finished, reward: -1.000000
resetting env. episode reward total was -19.000000. running mean: -20.829921
ep 27: game finished, reward: -1.000000
ep 27: game finished, reward: -1.000000
ep 27: game finished, reward: -1.000000
ep 27: game finished, reward: -1.000000
ep 27: game finished, reward: -1.000000
ep 27: game finished, reward: -1.000000
ep 27: game finished, reward: -1.000000
ep 27: game finished, reward: -1.000000
ep 27: game finished, reward: -1.00

ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: 1.000000 !!!!!!!!
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
ep 35: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.785623
ep 36: game finished, reward: -1.000000
ep 36: game finished, reward: -1.000000
ep 36: game finished, reward: -1.00

...

ep 275: game finished, reward: -1.000000
ep 275: game finished, reward: -1.000000
ep 275: game finished, reward: -1.000000
ep 275: game finished, reward: -1.000000
ep 275: game finished, reward: -1.000000
ep 275: game finished, reward: -1.000000
ep 275: game finished, reward: -1.000000
ep 275: game finished, reward: -1.000000
ep 275: game finished, reward: -1.000000
ep 275: game finished, reward: -1.000000
resetting env. episode reward total was -19.000000. running mean: -20.438782
ep 276: game finished, reward: -1.000000
ep 276: game finished, reward: -1.000000
ep 276: game finished, reward: -1.000000
ep 276: game finished, reward: -1.000000
ep 276: game finished, reward: -1.000000
ep 276: game finished, reward: -1.000000
ep 276: game finished, reward: -1.000000
ep 276: game finished, reward: -1.000000
ep 276: game finished, reward: -1.000000
ep 276: game finished, reward: -1.000000
ep 276: game finished, reward: -1.000000
ep 276: game finished, reward: -1.000000
ep 276: game finished

ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: 1.000000 !!!!!!!!
ep 284: game finished, reward: 1.000000 !!!!!!!!
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
ep 284: game finished, reward: -1.000000
resetting env. episode reward total was -19.000000. running mean: -20.438976
ep 285: game finished, reward: -1.000000
ep 285: game finished, reward: 1.000000 !!!!!!

ep 292: game finished, reward: -1.000000
ep 292: game finished, reward: -1.000000
ep 292: game finished, reward: 1.000000 !!!!!!!!
ep 292: game finished, reward: -1.000000
ep 292: game finished, reward: -1.000000
ep 292: game finished, reward: -1.000000
ep 292: game finished, reward: -1.000000
ep 292: game finished, reward: -1.000000
ep 292: game finished, reward: -1.000000
ep 292: game finished, reward: -1.000000
ep 292: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.415918
ep 293: game finished, reward: -1.000000
ep 293: game finished, reward: -1.000000
ep 293: game finished, reward: -1.000000
ep 293: game finished, reward: -1.000000
ep 293: game finished, reward: -1.000000
ep 293: game finished, reward: -1.000000
ep 293: game finished, reward: -1.000000
ep 293: game finished, reward: -1.000000
ep 293: game finished, reward: -1.000000
ep 293: game finished, reward: -1.000000
ep 293: game finished, reward: -1.000000
ep 293: game 

ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
ep 301: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.446828
ep 302: game finished, reward: -1.000000
ep 302: game finished, reward: -1.000000
ep 302: game finished, reward: -1.000000
ep 302: game finished

KeyboardInterrupt: 

In [9]:
[k for k , v in {1:2}.items()]

[1]

Discrete(6)
(210, 160, 3)
